# LSTM 

## Outline
1. Imports
2. Data
3. Model
5. Train
6. Predict

## Imports

In [1]:
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Data

### Ingestion 

In [2]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
print("Parent Directory", parentDir)
# Save the path to all of the datasets
india_cases_path = os.path.join(parentDir, "cleaned_datasets/india/daily_cases_india.csv")
india_vacc_path = os.path.join(parentDir, "cleaned_datasets/india/daily_vacc_india.csv")
usa_cases_path = os.path.join(parentDir, "cleaned_datasets/usa/daily_cases_usa.csv")
usa_vacc_path = os.path.join(parentDir, "cleaned_datasets/usa/vacc_usa.csv")

# Quick check to make sure the path exists
print("Path:", india_cases_path)
print("Exists:", os.path.exists(india_cases_path))

Parent Directory /deep-co-training
Path: /deep-co-training/cleaned_datasets/india/daily_cases_india.csv
Exists: True


In [3]:
# Load the data as a pandas dataframe
india_cases_df = pd.read_csv(india_cases_path)
india_vacc_df =  pd.read_csv(india_vacc_path)

usa_cases_df = pd.read_csv(usa_cases_path)
usa_vacc_df = pd.read_csv(usa_vacc_path)

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('India Vacc:\n',india_vacc_df.head(),'\n')

print('USA Cases:\n',usa_cases_df.head(),'\n')
print('USA Vacc:\n',usa_vacc_df.head(),'\n')

India Cases:
          Date  Confirmed  Deaths  Recovered  Active
0  2020-01-30        NaN     NaN        NaN     NaN
1  2020-01-31        0.0     0.0        0.0     0.0
2  2020-02-01        0.0     0.0        0.0     0.0
3  2020-02-02        1.0     0.0        0.0     0.0
4  2020-02-03        1.0     0.0        0.0     0.0 

India Vacc:
    Updated On  Total_Doses  First_Dose  Second_Dose
0  2021-01-16          NaN         NaN          NaN
1  2021-01-17      20656.0     20656.0          0.0
2  2021-01-18      81690.0     81690.0          0.0
3  2021-01-19     192152.0    192152.0          0.0
4  2021-01-20     111510.0    111510.0          0.0 

USA Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-04-12        NaN     NaN        NaN
1  2020-04-13    25322.0  1546.0    11785.0
2  2020-04-14    26713.0  2305.0     6484.0
3  2020-04-15    29380.0  2478.0     6093.0
4  2020-04-16    31542.0  4616.0     5234.0 

USA Vacc:
          date  total_doses  people_vacc  people_fully_va

### Pre-Processing

In [4]:
# Select only the Confirmed column for univariate analysis
# Selecting from the first index because the 0th index is NaN
india_cases_df = india_cases_df[["Confirmed"]][1:]
usa_cases_df = usa_cases_df[["Confirmed"]][1:]

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

India Cases:
    Confirmed
1        0.0
2        0.0
3        1.0
4        1.0
5        0.0 

USA Cases:
    Confirmed
1    25322.0
2    26713.0
3    29380.0
4    31542.0
5    32022.0 



In [5]:
# Normalize the data
india_cases_mean = india_cases_df.mean()
india_cases_std = india_cases_df.std()

usa_cases_mean = usa_cases_df.mean()
usa_cases_std = usa_cases_df.std()


india_cases_df = (india_cases_df-india_cases_mean)/india_cases_std
usa_cases_df = (usa_cases_df-usa_cases_mean)/usa_cases_std

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

India Cases:
    Confirmed
1  -0.687995
2  -0.687995
3  -0.687983
4  -0.687983
5  -0.687995 

USA Cases:
    Confirmed
1  -0.817861
2  -0.797170
3  -0.757499
4  -0.725340
5  -0.718200 



In [6]:
# Create train test splits
india_cases_train, india_cases_test = train_test_split(india_cases_df, test_size=0.2)
india_vacc_train, india_vacc_test = train_test_split(india_vacc_df, test_size=0.2)

usa_cases_train, usa_cases_test = train_test_split(usa_cases_df, test_size=0.2)
usa_vacc_train, usa_vacc_test = train_test_split(usa_vacc_df, test_size=0.2)

# Visualize splits
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

India Cases:
      Confirmed
113  -0.605466
118  -0.596356
420   0.054845
124  -0.577156
355  -0.514392
..         ...
55   -0.686475
271  -0.136463
386  -0.512168
260   0.093723
489   0.997701

[496 rows x 1 columns] 

USA Cases:
      Confirmed
528   0.775194
493   1.165165
23   -0.836469
114  -0.366324
440  -1.066016
..         ...
239   1.654677
82   -0.385512
166  -0.403525
250   2.501734
302   0.083444

[438 rows x 1 columns] 



In [7]:
# Convert to numpy array
india_cases_train, india_cases_test = india_cases_train.to_numpy().flatten(), india_cases_test.to_numpy()
usa_cases_train, usa_cases_test = usa_cases_train.to_numpy().flatten(), usa_cases_test.to_numpy()

# Visualize outputs
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

India Cases:
 [-6.05465670e-01 -5.96355767e-01  5.48452939e-02 -5.77155859e-01
 -5.14391762e-01 -3.47837589e-01 -6.87982552e-01 -1.93346184e-01
 -1.52483553e-01 -6.87995117e-01  3.71668903e-01 -6.87982552e-01
 -4.55259062e-01 -3.60779935e-01 -6.87831767e-01 -6.64460151e-01
 -6.87995117e-01 -5.77645909e-01  3.62374124e+00  4.05419526e-01
 -4.54291527e-01  9.05157440e-01  3.91714297e+00 -2.45040177e-01
  3.74747258e+00 -3.46518223e-01  6.13250989e-01 -1.08127745e-01
 -1.33723434e-01 -3.81977739e-01 -4.80616008e-01  8.94377979e-02
 -2.24747080e-01 -4.13793293e-01 -3.01320535e-01  8.18483055e-02
 -2.68612838e-01  3.40955169e+00  1.77759887e-01  4.36369708e-02
  2.26100014e+00 -6.87995117e-01  1.21203090e-01 -1.68001803e-01
  4.71676799e-01 -1.60688749e-01  1.82019552e-01 -5.63736028e-01
 -1.68768292e-01 -2.99209550e-01 -6.57913586e-01  3.66027045e-01
  1.49752737e+00 -2.02619438e-01 -4.51074789e-01 -6.87366848e-01
 -3.77479331e-01 -1.44779333e-02 -6.80393059e-01 -1.18129791e-01
 -2.5570818

In [8]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [9]:
# choose a number of time steps
n_steps = 3
# split into samples
india_cases_train_X, india_cases_train_y = split_sequence(india_cases_train, n_steps)
india_cases_test_X, india_cases_test_y = split_sequence(india_cases_test, n_steps)


# summarize the data
for i in range(0,5):
    print(india_cases_train_X[i], india_cases_train_y[i])

[-0.60546567 -0.59635577  0.05484529] -0.5771558587770755
[-0.59635577  0.05484529 -0.57715586] -0.5143917623597827
[ 0.05484529 -0.57715586 -0.51439176] -0.34783758858376324
[-0.57715586 -0.51439176 -0.34783759] -0.687982551549953
[-0.51439176 -0.34783759 -0.68798255] -0.19334618408914545


In [10]:
# Reshape the data into [samples, timesteps, features]
# univariate
n_features = 1

india_cases_train_X = india_cases_train_X.reshape((india_cases_train_X.shape[0], 
                                                   india_cases_train_X.shape[1], n_features))
india_cases_test_X = india_cases_test_X.reshape((india_cases_test_X.shape[0], 
                                                 india_cases_test_X.shape[1], n_features))

# Visualize outputs
print('India Cases:\n',india_cases_train_X,'\n')

India Cases:
 [[[-0.60546567]
  [-0.59635577]
  [ 0.05484529]]

 [[-0.59635577]
  [ 0.05484529]
  [-0.57715586]]

 [[ 0.05484529]
  [-0.57715586]
  [-0.51439176]]

 ...

 [[-0.6871658 ]
  [-0.68647471]
  [-0.13646269]]

 [[-0.68647471]
  [-0.13646269]
  [-0.51216769]]

 [[-0.13646269]
  [-0.51216769]
  [ 0.09372259]]] 



## Model 

### Univariate non-stacked

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [12]:
# Define the model
model_uni = Sequential()
model_uni.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model_uni.add(Dense(1))
model_uni.compile(optimizer='adam',loss='mse')
model_uni.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


### Univariate stacked 

In [15]:
# Define the model
model_uni_stacked = Sequential()
model_uni_stacked.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model_uni_stacked.add(LSTM(50, activation='relu'))
model_uni_stacked.add(Dense(1))
model_uni_stacked.compile(optimizer='adam',loss='mse')
model_uni_stacked.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 3, 50)             10400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


## Train 

In [17]:
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Univariate non-stacked

In [20]:
# Fit the model
model_uni.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

Epoch 1/500
16/16 [==============================] - 0s 19ms/step - loss: 0.8336 - val_loss: 0.9653
Epoch 2/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8424 - val_loss: 0.9868
Epoch 3/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8355 - val_loss: 1.0087
Epoch 4/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8374 - val_loss: 0.9847
Epoch 5/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8344 - val_loss: 0.9965
Epoch 6/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8271 - val_loss: 0.9825
Epoch 7/500
16/16 [==============================] - 0s 5ms/step - loss: 0.8296 - val_loss: 1.0078
Epoch 8/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8464 - val_loss: 1.0023
Epoch 9/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8332 - val_loss: 0.9617
Epoch 10/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8475 - val_loss: 1.0040
Epoch 11

16/16 [==============================] - 0s 6ms/step - loss: 0.8108 - val_loss: 1.0346
Epoch 84/500
16/16 [==============================] - 0s 5ms/step - loss: 0.8049 - val_loss: 1.0263
Epoch 85/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8144 - val_loss: 1.0133
Epoch 86/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8060 - val_loss: 1.0414
Epoch 87/500
16/16 [==============================] - 0s 5ms/step - loss: 0.8109 - val_loss: 1.0121
Epoch 88/500
16/16 [==============================] - 0s 5ms/step - loss: 0.8161 - val_loss: 1.0172
Epoch 89/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8072 - val_loss: 1.0277
Epoch 90/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8003 - val_loss: 1.0268
Epoch 91/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8191 - val_loss: 1.0312
Epoch 92/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8324 - val_loss: 1.0243
Epoch 93/500


16/16 [==============================] - 0s 6ms/step - loss: 0.7938 - val_loss: 1.0423
Epoch 165/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8159 - val_loss: 1.0623
Epoch 166/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8076 - val_loss: 1.1328
Epoch 167/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8194 - val_loss: 1.0702
Epoch 168/500
16/16 [==============================] - 0s 7ms/step - loss: 0.7891 - val_loss: 1.0187
Epoch 169/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7954 - val_loss: 1.0908
Epoch 170/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7972 - val_loss: 1.0513
Epoch 171/500
16/16 [==============================] - 0s 6ms/step - loss: 0.8012 - val_loss: 1.0240
Epoch 172/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7956 - val_loss: 1.0160
Epoch 173/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7881 - val_loss: 1.0821
Epoc

16/16 [==============================] - 0s 6ms/step - loss: 0.7667 - val_loss: 1.0563
Epoch 246/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7725 - val_loss: 1.0517
Epoch 247/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7702 - val_loss: 1.1168
Epoch 248/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7835 - val_loss: 1.0539
Epoch 249/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7776 - val_loss: 1.0839
Epoch 250/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7709 - val_loss: 1.0656
Epoch 251/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7815 - val_loss: 1.1099
Epoch 252/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7864 - val_loss: 1.0808
Epoch 253/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7675 - val_loss: 1.0780
Epoch 254/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7695 - val_loss: 1.0644
Epoc

16/16 [==============================] - 0s 6ms/step - loss: 0.7732 - val_loss: 1.0734
Epoch 327/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7541 - val_loss: 1.1012
Epoch 328/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7575 - val_loss: 1.0657
Epoch 329/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7464 - val_loss: 1.1175
Epoch 330/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7502 - val_loss: 1.0819
Epoch 331/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7464 - val_loss: 1.1489
Epoch 332/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7819 - val_loss: 1.0863
Epoch 333/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7708 - val_loss: 1.0953
Epoch 334/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7729 - val_loss: 1.0851
Epoch 335/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7542 - val_loss: 1.1027
Epoc

16/16 [==============================] - 0s 6ms/step - loss: 0.7430 - val_loss: 1.0840
Epoch 408/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7311 - val_loss: 1.1447
Epoch 409/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7392 - val_loss: 1.1453
Epoch 410/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7400 - val_loss: 1.1171
Epoch 411/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7431 - val_loss: 1.0946
Epoch 412/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7547 - val_loss: 1.1188
Epoch 413/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7297 - val_loss: 1.1327
Epoch 414/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7388 - val_loss: 1.0831
Epoch 415/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7331 - val_loss: 1.1542
Epoch 416/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7688 - val_loss: 1.1506
Epoc

16/16 [==============================] - 0s 6ms/step - loss: 0.7168 - val_loss: 1.1481
Epoch 489/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7184 - val_loss: 1.1225
Epoch 490/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7245 - val_loss: 1.1734
Epoch 491/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7289 - val_loss: 1.1654
Epoch 492/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7292 - val_loss: 1.1294
Epoch 493/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7221 - val_loss: 1.1560
Epoch 494/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7332 - val_loss: 1.1865
Epoch 495/500
16/16 [==============================] - 0s 6ms/step - loss: 0.7337 - val_loss: 1.1395
Epoch 496/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7324 - val_loss: 1.1300
Epoch 497/500
16/16 [==============================] - 0s 5ms/step - loss: 0.7347 - val_loss: 1.1038
Epoc

### Univariate stacked 

In [ ]:
# Fit the model
model_uni_stacked.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

Epoch 1/500
16/16 [==============================] - 2s 42ms/step - loss: 1.0392 - val_loss: 0.8707
Epoch 2/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0387 - val_loss: 0.8709
Epoch 3/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0384 - val_loss: 0.8701
Epoch 4/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0385 - val_loss: 0.8707
Epoch 5/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0381 - val_loss: 0.8703
Epoch 6/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0381 - val_loss: 0.8703
Epoch 7/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0381 - val_loss: 0.8705
Epoch 8/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0379 - val_loss: 0.8709
Epoch 9/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0376 - val_loss: 0.8710
Epoch 10/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0377 - val_loss: 0.8706
Epoch 

16/16 [==============================] - 0s 10ms/step - loss: 1.0153 - val_loss: 0.8919
Epoch 83/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0198 - val_loss: 0.8962
Epoch 84/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0166 - val_loss: 0.8807
Epoch 85/500
16/16 [==============================] - 0s 9ms/step - loss: 1.0124 - val_loss: 0.8943
Epoch 86/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0141 - val_loss: 0.8904
Epoch 87/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0104 - val_loss: 0.9004
Epoch 88/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0127 - val_loss: 0.8930
Epoch 89/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0090 - val_loss: 0.8981
Epoch 90/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0130 - val_loss: 0.8993
Epoch 91/500
16/16 [==============================] - 0s 10ms/step - loss: 1.0067 - val_loss: 0.9047
Epoc

Epoch 163/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9654 - val_loss: 0.9118
Epoch 164/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9662 - val_loss: 0.9192
Epoch 165/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9678 - val_loss: 0.9155
Epoch 166/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9636 - val_loss: 0.9202
Epoch 167/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9631 - val_loss: 0.9162
Epoch 168/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9635 - val_loss: 0.9235
Epoch 169/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9613 - val_loss: 0.9126
Epoch 170/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9622 - val_loss: 0.9230
Epoch 171/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9606 - val_loss: 0.9125
Epoch 172/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9688 - 

16/16 [==============================] - 0s 9ms/step - loss: 0.9277 - val_loss: 0.9329
Epoch 244/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9327 - val_loss: 0.9254
Epoch 245/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9313 - val_loss: 0.9369
Epoch 246/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9281 - val_loss: 0.9341
Epoch 247/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9254 - val_loss: 0.9495
Epoch 248/500
16/16 [==============================] - 0s 10ms/step - loss: 0.9346 - val_loss: 0.9417
Epoch 249/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9355 - val_loss: 0.9660
Epoch 250/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9322 - val_loss: 0.9260
Epoch 251/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9223 - val_loss: 0.9506
Epoch 252/500
16/16 [==============================] - 0s 9ms/step - loss: 0.9194 - val_loss: 0.9362
Ep

Epoch 324/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8880 - val_loss: 0.9477
Epoch 325/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8745 - val_loss: 0.9537
Epoch 326/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8670 - val_loss: 0.9587
Epoch 327/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8753 - val_loss: 0.9970
Epoch 328/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8769 - val_loss: 0.9741
Epoch 329/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8781 - val_loss: 0.9541
Epoch 330/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8650 - val_loss: 0.9973
Epoch 331/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8642 - val_loss: 0.9431
Epoch 332/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8682 - val_loss: 1.0190
Epoch 333/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8773

16/16 [==============================] - 0s 10ms/step - loss: 0.8035 - val_loss: 0.9656
Epoch 405/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8027 - val_loss: 0.9936
Epoch 406/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8139 - val_loss: 0.9379
Epoch 407/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8167 - val_loss: 0.9282
Epoch 408/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8366 - val_loss: 1.0561
Epoch 409/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8194 - val_loss: 0.9670
Epoch 410/500
16/16 [==============================] - 0s 9ms/step - loss: 0.8183 - val_loss: 0.9616
Epoch 411/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8206 - val_loss: 0.9742
Epoch 412/500
16/16 [==============================] - 0s 10ms/step - loss: 0.8105 - val_loss: 1.0015
Epoch 413/500
16/16 [==============================] - 0s 10ms/step - loss: 0.7948 - val_loss: 0.

### Univariate stacked 

## Predict

In [ ]:
x_input = np.array([india_cases_test_X[25]])
print(x_input)
yhat = model.predict(x_input)
print(yhat)